In [4]:
#Use this window to import packages you will use later
#When you first start working or import a new package, this cell needs to be run before doing anything else.


import numpy as np
import pylab
import matplotlib
import matplotlib.gridspec as gridspec

import scipy

import astropy.io.fits as fits

from astropy.wcs import WCS
from astropy import units as u

import regions

import pyspeckit as psk

from spectral_cube import SpectralCube

from reproject import reproject_interp
from reproject.mosaicking import find_optimal_celestial_wcs 

# Suppress warnings we don't care about:
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
    
# Embeds plots in the Jupyter window 
%matplotlib inline                             

matplotlib.rc('text', usetex=True)
#matplotlib.rc('font', family='sans-serif')
font = {'family' : 'sans-serif',
        'weight' : 'bold',
        'size'   : 22}
matplotlib.rc('font', **font)

datadir = "../data/"

#print("all imported")

all imported


In [2]:
# A useful exercise is to compare data from different species. First, we want to compare 13CO and 12CO 
# (and eventually C18O, and C17O)
# Here, we demonstrate a method to match shapes of 2D images (like moment maps) so we can calculate ratios.
        
sc13 = SpectralCube.read(datadir + '')    
print(sc13)                                              

sc12 = SpectralCube.read('H8_12C16O.cube.pbcor.fits')    # Comparing to the 13 CO,
print(sc12)                                              # here we immediately see this image is a different shape.
                                                        # In order to compare the two, we have to make them the same.

    


sc13_kms = sc13.with_spectral_unit(u.km/u.s,velocity_convention="radio")
sc13_kms.allow_huge_operations=True
sc13_K_kms = sc13_kms.to(u.K)
sc12_kms = sc12.with_spectral_unit(u.km/u.s,velocity_convention="radio")
sc12_kms.allow_huge_operations=True
sc12_K_kms = sc12_kms.to(u.K)

sc13_moment0  = sc13_K_kms.moment(order=0,how='slice')    # Calculate the first Moment 0 map 
sc12_moment0  = sc12_K_kms.moment(order=0,how='slice')    # Calculate the second Moment 0 map 


sc13_moment0_reproject, footprint = reproject_interp(sc13_moment0.hdu,sc12_moment0.header)

                                                        # Now that images are same size, compute ratio of moment maps
ratio_12CO_13CO = sc12_moment0.hdu.data/sc13_moment0_reproject

badpix = pylab.where(sc13_moment0_reproject<50) # Identify emission below a threshold to mask
badpix2 = pylab.where(sc12_moment0.hdu.data<0)           # Mask 12CO absorption
ratio_12CO_13CO[badpix] = np.nan                        # Mask the ratio map
ratio_12CO_13CO[badpix2] = np.nan

fig1 = pylab.figure(1,figsize=(15,15))

ax1 = pylab.subplot(2,2,1,projection=sc12_moment0.wcs) 
im1 = pylab.imshow(ratio_12CO_13CO,cmap='copper')
RA = ax1.coords[0]                                                                  
Dec = ax1.coords[1]
RA.set_ticks(size=-3)                                                                                      
Dec.set_ticks(size=-3)                                                                                    
pylab.xlabel('Right Ascension',fontsize=20,labelpad=1)                               
pylab.ylabel('Declination',fontsize=20,labelpad=0)
ax1.tick_params(axis = 'both', which = 'major', labelsize = 15)    
cb=pylab.colorbar(im1,fraction=0.046,pad=0.04)                                      
cb.set_label(label='12CO/13CO Ratio',fontsize=20,rotation=270,labelpad=20) 
cb.ax.tick_params(which = 'major', labelsize = 10)   
pylab.annotate(s='$^{12}$CO/$^{13}$CO',fontsize=35,xy=(0.02,0.91),xycoords="axes fraction");  

testing 2
